In [2]:
!pip install tensorflow==2.15

In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.15.0'

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [7]:
!kaggle datasets download -d vijaygiitk/multiclass-weather-dataset

Dataset URL: https://www.kaggle.com/datasets/vijaygiitk/multiclass-weather-dataset
License(s): CC0-1.0
 94% 127M/134M [00:01<00:00, 107MB/s] 
100% 134M/134M [00:01<00:00, 82.4MB/s]


In [8]:
!unzip '/content/multiclass-weather-dataset.zip'

Archive:  /content/multiclass-weather-dataset.zip
  inflating: dataset/alien_test/Cloud_1.png  
  inflating: dataset/alien_test/Cloud_2.jpg  
  inflating: dataset/alien_test/Cloud_3.jpeg  
  inflating: dataset/alien_test/Cloud_4.jpg  
  inflating: dataset/alien_test/foggy_1.jpg  
  inflating: dataset/alien_test/foggy_10.jpg  
  inflating: dataset/alien_test/foggy_2.jpg  
  inflating: dataset/alien_test/foggy_3.jpg  
  inflating: dataset/alien_test/foggy_4.jpg  
  inflating: dataset/alien_test/foggy_5.jpg  
  inflating: dataset/alien_test/foggy_6.jpg  
  inflating: dataset/alien_test/foggy_7.jpg  
  inflating: dataset/alien_test/foggy_8.jpg  
  inflating: dataset/alien_test/foggy_9.jpg  
  inflating: dataset/alien_test/rain_1.jpg  
  inflating: dataset/alien_test/rain_2.png  
  inflating: dataset/alien_test/rain_3.jpg  
  inflating: dataset/alien_test/rain_4.jpg  
  inflating: dataset/alien_test/rain_5.jpg  
  inflating: dataset/alien_test/rain_6.jpg  
  inflating: dataset/alien_test/sh

In [10]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import ImageFile

In [11]:
import os
import shutil
import random

def split_dataset(dataset_dir, test_ratio=0.2):
    categories = ['cloudy', 'foggy', 'rainy', 'shine', 'sunrise']
    for category in categories:
        category_path = os.path.join(dataset_dir, category)
        train_path = os.path.join(dataset_dir, 'train', category)
        test_path = os.path.join(dataset_dir, 'test', category)
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(test_path, exist_ok=True)
        files = os.listdir(category_path)
        random.shuffle(files)
        split_index = int(len(files) * (1 - test_ratio))
        train_files = files[:split_index]
        test_files = files[split_index:]
        for file in train_files:
            shutil.move(os.path.join(category_path, file), os.path.join(train_path, file))
        for file in test_files:
            shutil.move(os.path.join(category_path, file), os.path.join(test_path, file))

dataset_dir = '/content/dataset'
split_dataset(dataset_dir)


In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=[0.99, 1.01],
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    data_format="channels_last",
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
training_set = train_datagen.flow_from_directory(
    '/content/dataset/train',
    target_size=(180, 180),
    batch_size=64,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    '/content/dataset/test',
    target_size=(180, 180),
    batch_size=64,
    class_mode='categorical'
)

Found 1200 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


In [14]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(180, 180, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [15]:
for layer in base_model.layers:
    layer.trainable = False

In [16]:
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [17]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 180, 180, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 180, 180, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 180, 180, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 90, 90, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 90, 90, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 90, 90, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 45, 45, 128)       0     

In [18]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [27]:
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=20,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

Epoch 1/20
19/19 [==============================] - 18s 892ms/step - loss: 0.0400 - accuracy: 0.9933 - val_loss: 0.2201 - val_accuracy: 0.9267
Epoch 2/20
19/19 [==============================] - 16s 853ms/step - loss: 0.0343 - accuracy: 0.9958 - val_loss: 0.2215 - val_accuracy: 0.9300
Epoch 3/20
19/19 [==============================] - 16s 864ms/step - loss: 0.0269 - accuracy: 0.9958 - val_loss: 0.2360 - val_accuracy: 0.9233
Epoch 4/20
19/19 [==============================] - 18s 952ms/step - loss: 0.0276 - accuracy: 0.9950 - val_loss: 0.2220 - val_accuracy: 0.9300
Epoch 5/20
19/19 [==============================] - 16s 860ms/step - loss: 0.0256 - accuracy: 0.9958 - val_loss: 0.2227 - val_accuracy: 0.9233
Epoch 6/20
19/19 [==============================] - 16s 856ms/step - loss: 0.0250 - accuracy: 0.9958 - val_loss: 0.2525 - val_accuracy: 0.9233
Epoch 7/20
19/19 [==============================] - 17s 872ms/step - loss: 0.0208 - accuracy: 0.9975 - val_loss: 0.2665 - val_accuracy: 0.9300

In [28]:

loss, accuracy = model.evaluate(
    test_set,
    steps=len(test_set),
    verbose=2,
    use_multiprocessing=True,
    workers=2
)

print(f'Model performance on test images: \nAccuracy = {accuracy}\nLoss = {loss}')

model.save('vgg16_model.h5')


5/5 - 2s - loss: 0.2313 - accuracy: 0.9367 - 2s/epoch - 365ms/step
Model performance on test images: 
Accuracy = 0.9366666674613953
Loss = 0.23134218156337738


In [29]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model('/content/vgg16_model.h5')

img_path = '/content/dataset/test/shine/shine124.jpg'
img = image.load_img(img_path, target_size=(180, 180))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
pred = np.argmax(preds, axis=1)

index = ['cloudy', 'foggy', 'rainy', 'shine', 'sunrise']

result = index[pred[0]]
print(f'The model predicts the image is: {result}')


1/1 [==============================] - 0s 143ms/step
The model predicts the image is: sunrise
